In [1]:
import pandas as pd
import re
import numpy as np

pd.set_option('display.max_rows', 20)


In [2]:
df = pd.read_csv('monster_com-job_sample.csv')

In [3]:
df

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,United States of America,US,NaN,No,jobs.monster.com,This is a major premier Cincinnati based finan...,Assistant Vice President - Controller Job in C...,Full Time,"Cincinnati, OH",NaN,http://jobview.monster.com/Assistant-Vice-Pres...,"120,000.00 - 160,000.00 $ /yearbonus",NaN,a80bc8cc3a90c17eef418963803bc640
21996,United States of America,US,NaN,No,jobs.monster.com,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,http://jobview.monster.com/Accountant-Job-Cinc...,"45,000.00 - 60,000.00 $ /year",Manager (Manager/Supervisor of Staff),419a3714be2b30a10f628de207d041de
21997,United States of America,US,NaN,No,jobs.monster.com,RE: Adobe AEM- Client - Loca...,AEM/CQ developer Job in Chicago,Full Time,"Chicago, IL 60602",NaN,http://jobview.monster.com/AEM-CQ5-developer-J...,NaN,NaN,5a590350b73b2cec46b05750a208e345
21998,United States of America,US,NaN,No,jobs.monster.com,Jernberg Industries was established in 1937 an...,Electrician - Experienced Forging Electrician ...,Full Time Employee,"Chicago, IL 60609","Jernberg Industries, Inc.",http://jobview.monster.com/Electrician-Experie...,25.00 - 28.00 $ /hour,Installation/Maintenance/Repair,40161cf61c283af9dc2b0a62947a5f1b


In [4]:
# Shape of data
df.shape

(22000, 14)

# Country

In [5]:
df['country'].value_counts()

United States of America    22000
Name: country, dtype: int64

# Country_code

In [6]:
df['country_code'].value_counts()

US    22000
Name: country_code, dtype: int64

# has_expired

In [7]:
df['has_expired'].value_counts()

No    22000
Name: has_expired, dtype: int64

# job_board

In [8]:
df['job_board'].value_counts()

jobs.monster.com    22000
Name: job_board, dtype: int64

# date_added

In [9]:
df['date_added'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 22000 entries, 0 to 21999
Series name: date_added
Non-Null Count  Dtype 
--------------  ----- 
122 non-null    object
dtypes: object(1)
memory usage: 172.0+ KB


# Dropping unnecessary column

In [10]:
df.drop(['country', 'country_code', 'has_expired', 'job_board', 'date_added', 'uniq_id'], axis=1, inplace=True)

In [11]:
df.head()

,job_description,job_title,job_type,location,organization,page_url,salary,sector
0,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development
1,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN
2,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN
3,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager)
4,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management


# job_type

In [12]:
df1 = df['job_type'].value_counts().to_frame().reset_index()
df1

,index,job_type
0,Full Time,6757
1,Full Time Employee,6617
2,"Full Time, Employee",3360
3,Full Time Temporary/Contract/Project,1062
4,"Full Time, Temporary/Contract/Project",533
...,...,...
34,Full Time / Temporary/Contract/Project,1
35,"Part Time, Intern",1
36,"Part Time , Temporary/Contract/Project",1
37,Exempt,1


# Devide job_type column in two part
    job_type_1 (full time, part_time)
    job_type_2 (Employee, Temporary/Contract/Project, intern)

In [13]:
job_type_1 = df1['index'].apply(lambda x : re.findall(r'\w{3,4}\s\w{4}', x))

job_type_1 = job_type_1.apply(lambda x: x[1] if len(x) == 2 else x)

df['job_type_1'] = job_type_1.apply(lambda x: ' '.join(x)).apply(lambda x: x.replace(' ', ''))

df['job_type_1']

0        FullTime
1        FullTime
2        FullTime
3        FullTime
4        FullTime
           ...   
21995         NaN
21996         NaN
21997         NaN
21998         NaN
21999         NaN
Name: job_type_1, Length: 22000, dtype: object

In [14]:
job_type_2 = df1['index'].apply(lambda x: x.split()[-1] if x.split()[-1] not in ('Time', 'Diem') else np.nan)

df['job_type_2'] = job_type_2

df['job_type_2']

0                               NaN
1                          Employee
2                          Employee
3        Temporary/Contract/Project
4        Temporary/Contract/Project
                    ...            
21995                           NaN
21996                           NaN
21997                           NaN
21998                           NaN
21999                           NaN
Name: job_type_2, Length: 22000, dtype: object

In [15]:
df.drop('job_type', axis=1, inplace=True)

In [16]:
df.head()

,job_description,job_title,location,organization,page_url,salary,sector,job_type_1,job_type_2
0,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,FullTime,NaN
1,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,FullTime,Employee
2,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,FullTime,Employee
3,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),FullTime,Temporary/Contract/Project
4,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,FullTime,Temporary/Contract/Project


# location

In [17]:
df.location

0                                        Madison, WI 53702
1                                        Madison, WI 53708
2        DePuy Synthes Companies is a member of Johnson...
3                                                Dixon, CA
4                                             Camphill, PA
                               ...                        
21995                                       Cincinnati, OH
21996                                 Cincinnati, OH 45236
21997                                    Chicago, IL 60602
21998                                    Chicago, IL 60609
21999                                       Cincinnati, OH
Name: location, Length: 22000, dtype: object

In [18]:
df['loc_'] = df.page_url.str.extract(r'-(?i)job-(.*?)(?i)-us-\d')

df['location_extract'] = df.loc_.str[:-3] + ', ' + df['loc_'].str[-2:]

df['location_extract'].sample(10)

15643                    hollywood, fl
7529                     Las-Vegas, NV
1496                    shady-dale, ga
8566                     highlands, co
2242                   parkersburg, wv
13414    Raleigh-NC-or-Columbia-SC, NC
10326                     Newville, PA
14112                  Minneapolis, MN
7907                       Madison, WI
726                               , ar
Name: location_extract, dtype: object

In [19]:
df.drop(['location', 'loc_'], axis=1, inplace=True)

# organization

In [20]:
df['organization'].unique()

array([nan, 'Printing and Publishing', 'Personal and Household Services',
       'Altec Industries', 'Retail', 'Computer/IT Services',
       'Computer Software',
       'Hotels and Lodging Personal and Household Services', 'Insurance',
       'Business Services - Other', 'Education',
       'Construction - Industrial Facilities and InfrastructureConstruction - Residential & Commercial/Office',
       'Accounting and Auditing Services', 'Legal Services',
       'Construction - Residential & Commercial/Office',
       'Engineering Services', 'AllComputer SoftwareComputer/IT Services',
       'Healthcare Services', 'Chicago, IL', 'Manufacturing - Other',
       'Oklahoma City, OK', 'Aerospace and Defense', 'San Francisco, CA',
       'Advertising and PR ServicesManagement Consulting ServicesBusiness Services - Other',
       'Other/Not Classified',
       'RetailAdvertising and PR ServicesBusiness Services - Other',
       'All', 'Electronics, Components, and Semiconductor Mfg',
       '

In [21]:
df['organization'].isnull().sum()

6867

Category : Other/Not Classified

In [22]:
other_index = df[df['organization'].str.match(r'(?i)Other\/Not Classified', na=False)].index
other_index

Int64Index([   42,    69,   133,   154,   208,   224,   230,   251,   273,
              295,
            ...
            21734, 21765, 21773, 21849, 21872, 21902, 21918, 21920, 21978,
            21984],
           dtype='int64', length=1048)

In [23]:
df.loc[other_index, 'organization'] = 'Other'

category City_names

In [24]:
ind = list(df[df.organization.str.contains(r'[\w\/ ]*\w*, \w{2}[\d]*$|[\w\/ ]*\w*, \w{2}[\d]*[^\w]')==True]['organization'].index)

In [25]:
ind

[29,
 36,
 38,
 73,
 152,
 249,
 425,
 624,
 652,
 674,
 734,
 766,
 776,
 817,
 924,
 1012,
 1019,
 1045,
 1049,
 1063,
 1070,
 1095,
 1106,
 1129,
 1139,
 1146,
 1175,
 1227,
 1239,
 1246,
 1251,
 1309,
 1327,
 1329,
 1332,
 1340,
 1354,
 1381,
 1439,
 1486,
 1512,
 1573,
 1590,
 1594,
 1622,
 1650,
 1660,
 1681,
 1690,
 1747,
 1762,
 1828,
 1870,
 1889,
 2052,
 2077,
 2098,
 2155,
 2233,
 2297,
 2376,
 2426,
 2563,
 2604,
 2631,
 2695,
 2750,
 2766,
 2783,
 2795,
 2807,
 3000,
 3047,
 3065,
 3077,
 3181,
 3187,
 3188,
 3229,
 3261,
 3281,
 3433,
 3434,
 3456,
 3660,
 3886,
 4036,
 4040,
 4087,
 4088,
 4115,
 4215,
 4217,
 4235,
 4241,
 4341,
 4354,
 4377,
 4421,
 4460,
 4506,
 4510,
 4516,
 4533,
 4534,
 4582,
 4588,
 4594,
 4647,
 4673,
 4682,
 4725,
 4755,
 5015,
 5065,
 5165,
 5395,
 5480,
 5706,
 5748,
 5854,
 6014,
 6781,
 6812,
 6845,
 6898,
 6899,
 6900,
 6910,
 6911,
 6929,
 7085,
 8484,
 8548,
 8623,
 8712,
 8767,
 8830,
 8896,
 8935,
 9511,
 10079,
 10137,
 10203,
 10217,


In [26]:
df.loc[ind, 'organization'] = 'Other'

In [27]:
df.organization.sample(10)

6876                                  Computer/IT Services
19447             Healthcare Services Other/Not Classified
20395                                  Healthcare Services
1728                                   Healthcare Services
16454    Travel, Transportation and TourismTransport an...
15743                                  Healthcare Services
17433                                                  All
5452                          Medical Devices and Supplies
16581                                               Retail
8291                    Nonprofit Charitable Organizations
Name: organization, dtype: object

# sector

In [28]:
df.sector.value_counts()

Experienced (Non-Manager)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [29]:
len(df.sector.unique())

164

In [30]:
df.sector.isnull().sum()

5194

In [31]:
ind = list(df[df.sector.apply(lambda x:len(str(x))>100)]['sector'].index)

In [32]:
df.loc[ind, 'sector'] = 'Other'

# Final Cleaned Data

In [33]:
df.head()

,job_description,job_title,organization,page_url,salary,sector,job_type_1,job_type_2,location_extract
0,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,FullTime,NaN,"madison, wi"
1,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,FullTime,Employee,"madison, wi"
2,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,FullTime,Employee,"raynham, ma"
3,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),FullTime,Temporary/Contract/Project,"dixon, ca"
4,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,FullTime,Temporary/Contract/Project,"camphill, pa"
